## Import libraries

In [77]:
import pandas as pd
import numpy as np
import os

## Variables to set
The result of MEASURE_MULTI x SAMPLES must be equal to number of rows of raw data (i.e. 2500)

In [78]:
SRCPATH = "../capture"
OUTPATH = "../capture/processed"

DATAFOLDERS = [
    "3225",             # CLASS 0
    "3225_REVERSO",     # CLASS 1
    "3225_VAZIO",       # CLASS 2
    "3230",             # CLASS 3
    "3230_VAZIO",       # CLASS 4
    "3235"              # CLASS 5
]
CLASSES = [0, 1, 2, 3, 4, 5]

MEASURE_MULTI = 100
SAMPLES = 25

## Load raw sensor data
Each raw dataset has 2500 rows and 7 columns (AcX, AcY, AcZ, GyX, GyY, GyZ, Tmp).

In [79]:
captures0 = []
captures1 = []
captures2 = []
captures3 = []
captures4 = []
captures5 = []
captures = [captures0, captures1, captures2, captures3, captures4, captures5]

for i, DATAFOLDER in enumerate(DATAFOLDERS):
    for _, _, DATAFILES in os.walk(SRCPATH+os.sep+DATAFOLDER):
        for DATAFILE in DATAFILES:
            df = pd.read_csv(SRCPATH+os.sep+DATAFOLDER+os.sep+DATAFILE)
            captures[i].append(df)
            
for j, datasets in enumerate(captures):            
    print("Datasets from class "+str(j)+": "+str(len(datasets))+" with shape ", datasets[0].shape)

Datasets from class 0: 10 with shape  (2500, 7)
Datasets from class 1: 10 with shape  (2500, 7)
Datasets from class 2: 3 with shape  (2500, 7)
Datasets from class 3: 10 with shape  (2500, 7)
Datasets from class 4: 1 with shape  (2500, 7)
Datasets from class 5: 10 with shape  (2500, 7)


## Make a new head for dataset

In [80]:
head = ["AcX", "AcY", "AcZ", "GyX", "GyY", "GyZ", "Tmp"]
list_head = []

for hd in head:        
    for i in range(MEASURE_MULTI):
        head_aux = hd + str(i)
        list_head.append(head_aux)
        
print(len(list_head))

700


## Split each dataset into 25 samples and save them
Each new dataset is composed by 25 samples with 100 values of each measure, i.e. 25 rows and 700 columns.

In [81]:
for Class, capture in enumerate(captures):
    for idx,dataset in enumerate(capture):   
        list_df = []
        for i in range(SAMPLES):
            lst = dataset.iloc[MEASURE_MULTI*i:MEASURE_MULTI*(i+1),:]
            lst = lst.transpose()
            lst = lst.values
            lst = np.resize(lst, (1,7*MEASURE_MULTI))    
            list_df.append(lst)
        new_df = np.concatenate([i for i in list_df], axis=0)
        new_df = pd.DataFrame(new_df, columns=list_head, index=None)
        new_df.to_csv(OUTPATH+os.sep+DATAFOLDERS[Class]+os.sep+str(idx)+"_"+str(SAMPLES)+"samples.csv", \
                      sep=",", index=False)
        list_df.clear()

## Load all saved datasets and concatenate them into one unique dataset
Each dataset row has a class according to was describe above.

In [82]:
list_newdfs = []
for i, DATAFOLDER in enumerate(DATAFOLDERS):
    for _, _, DATAFILES in os.walk(OUTPATH+os.sep+DATAFOLDER):
        for DATAFILE in DATAFILES:
            newdf = pd.read_csv(OUTPATH+os.sep+DATAFOLDER+os.sep+DATAFILE)
            class_col = [i] * 25
            newdf["Class"] = class_col
            list_newdfs.append(newdf.values)
final_df = np.concatenate([j for j in list_newdfs], axis=0)
list_head.append("Class")
final_df = pd.DataFrame(final_df, columns=list_head, index=None)
final_df.to_csv(OUTPATH+os.sep+"32FINAL.csv", sep=",", index=False)